Question1
Refer to the autocallable mecahnism/assumptions guided on this page:
https://www.morganstanley.com/structuredinvestments/downloads/Autocallable.pdf


In [101]:
##########
#Mingze Lin @2022
##########

import numpy
import scipy 
import math

#maturity in years
#knockin = principal barrier
#knockout = call hurdle

def autocallable(stockpr, knockin, knockout, coupon_rate, maturity, sigma, rounds):
  vol = sigma/math.sqrt(252)  
  sum = 0

  for round in range(rounds):
    s_new = stockpr 
    s_curr = stockpr
    nocoupon = 0
    payout = 0
    called_indicator = 0
    finished_year = 0
    for i in range(maturity*252):
        s_new += s_curr * vol * scipy.random.normal() 
        
        if i%252==251:
            if s_new >= knockout:
                called_indicator=1
                finished_year = math.ceil(i/252)
                break
            elif s_new < knockin:
                nocoupon += 1
        s_curr = s_new
    
    if s_curr < knockin: 
        payout += (maturity - nocoupon)* coupon_rate * stockpr 
        payout += s_curr
    elif called_indicator == 1:
        payout += (finished_year - nocoupon)* coupon_rate * stockpr 
        payout += stockpr
    else:
        payout += (maturity - nocoupon)* coupon_rate * stockpr 
        payout += stockpr
    sum += payout
  print("avg payout = " + str(sum/rounds))
  print("option price = " + str(sum/rounds-stockpr))

In [102]:
#knock-in = 60
#knock-out = 110
autocallable(100, 60, 110, 0.02, 5, 0.16, 10000)

avg payout = 101.59877873272171
option price = 1.598778732721712


Monte Carlo Simulation for Basket Trade

Call Pricer on avg of worst 3

In [53]:
#maturity in years
def avgBasket(stockpr, strike, maturity, sigma, rounds, corr12, corr13, corr23):
    vol = sigma/math.sqrt(252)
    sum = 0
    price = 0
    for round in range(rounds):
        s1 = stockpr #s1
        s2 = stockpr
        s3 = stockpr
        s4 = stockpr #s2
        s5 = stockpr #s3
        for i in range(maturity*252):
            move1 = s1 * vol * scipy.random.normal()
            move2 = s2 * vol * scipy.random.normal()
            move3 = s3 * vol * scipy.random.normal()
            s1 += move1 
            s2 += move2
            s3 += move3
            
            move4 = corr12*move1 + math.sqrt(1-corr12*corr12)*move2
            s4 += move4
            
            move5 = corr13*move1+corr23*move4+ math.sqrt(1-(corr13*corr13)-(corr23*corr23))*move3
            s5 += move5
        
        avg = (s1+s4+s5)/3
        sum += avg
        price += max(avg-strike, 0)
        
    print("stock price = " + str(sum/rounds))
    print("option price = " + str(price/rounds))

In [54]:
avgBasket(100, 100, 1, 0.16, 10000, 0.1, 0.1, 0.1)
#correlations + 50%, +70%, +80%
avgBasket(100, 100, 1, 0.16, 10000, 0.1*1.5, 0.1*1.5, 0.1*1.5)
avgBasket(100, 100, 1, 0.16, 10000, 0.1*1.7, 0.1*1.7, 0.1*1.7)
avgBasket(100, 100, 1, 0.16, 10000, 0.1*1.8, 0.1*1.8, 0.1*1.8)

stock price = 100.05032837798235
option price = 4.108710124469348
stock price = 99.85889499088735
option price = 4.145536639848335
stock price = 100.07375065433473
option price = 4.414628045695976
stock price = 100.11098059165685
option price = 4.429556651244858


In [14]:
#check the sensitivity to correlation
avgBasket(100, 100, 1, 0.16, 10000, 0.11, 0.1, 0.1)
avgBasket(100, 100, 1, 0.16, 10000, 0.11, 0.11, 0.11)
#########
#Conclusion: avg of short correlation
#########

stock price = 100.12733747628931
option price = 4.145442045240576
stock price = 100.19087725425523
option price = 4.195444179957302


Put Pricer on worst 3

In [12]:
#maturity in years
def worstBasket(stockpr, strike, maturity, sigma, rounds, corr12, corr13, corr23):
    vol = sigma/math.sqrt(252)
    sum = 0
    price = 0
    for round in range(rounds):
        s1 = stockpr #s1
        s2 = stockpr
        s3 = stockpr
        s4 = stockpr #s2
        s5 = stockpr #s3
        for i in range(maturity*252):
            move1 = s1 * vol * scipy.random.normal()
            move2 = s2 * vol * scipy.random.normal()
            move3 = s3 * vol * scipy.random.normal()
            s1 += move1 
            s2 += move2
            s3 += move3
            
            move4 = corr12*move1 + math.sqrt(1-corr12*corr12)*move2
            s4 += move4
            
            move5 = corr13*move1+corr23*move4+ math.sqrt(1-(corr13*corr13)-(corr23*corr23))*move3
            s5 += move5
        
        minimum = min(min(s1,s4),s5)
        sum += minimum
        price += max(strike - minimum, 0)
        
    print("stock price = " + str(sum/rounds))
    print("option price = " + str(price/rounds))

In [13]:
worstBasket(100, 60, 1, 0.16, 10000, 0.1, 0.1, 0.1)
#correlations + 50%, +70%, +80%
worstBasket(100, 60, 1, 0.16, 10000, 0.1*1.5, 0.1*1.5, 0.1*1.5)
worstBasket(100, 60, 1, 0.16, 10000, 0.1*1.7, 0.1*1.7, 0.1*1.7)
worstBasket(100, 60, 1, 0.16, 10000, 0.1*1.8, 0.1*1.8, 0.1*1.8)

stock price = 87.77377750621005
option price = 0.008336872920836073
stock price = 87.80044528033036
option price = 0.010409998400261328
stock price = 88.17610263588351
option price = 0.007744412168993287
stock price = 87.96399273995227
option price = 0.009356753322541178


In [17]:
#check the sensitivity to correlation
worstBasket(100, 60, 1, 0.16, 10000, 0.11, 0.1, 0.1)
worstBasket(100, 60, 1, 0.16, 10000, 0.11, 0.11, 0.11)
#sensitivity to correlation after changing the strike to 90%
worstBasket(100, 90, 1, 0.16, 10000, 0.1, 0.1, 0.1)
worstBasket(100, 90, 1, 0.16, 10000, 0.11, 0.1, 0.1)
worstBasket(100, 90, 1, 0.16, 10000, 0.11, 0.11, 0.11)
#########
#Conclusion: worst of long correlation
#########

stock price = 87.5680190612218
option price = 0.00643220790915426
stock price = 87.45114616764786
option price = 0.008302502285701173
stock price = 87.6110372160221
option price = 5.681966553283787
stock price = 87.64954506743291
option price = 5.690015491666136
stock price = 87.7003264205062
option price = 5.623833097022666


Pricer for down-and-in put on worst of 3

In [103]:
#Assume pays 10% coupon every year as long as the worst of a basket of 3 names is above 80% of its initial level.
#maturity in years
counpon_rate = 0.1
coupon_threshold = 80
def knockinPutBasket(stockpr, strike, maturity, sigma, rounds, corr12, corr13, corr23):
    vol = sigma/math.sqrt(252)
    sum = 0
    price = 0
    for round in range(rounds):
        s1 = stockpr #s1
        s2 = stockpr
        s3 = stockpr
        s4 = stockpr #s2
        s5 = stockpr #s3
        knockin_indicator = 0
        coupon_num = 0
        for i in range(maturity*252):
            move1 = s1 * vol * scipy.random.normal()
            move2 = s2 * vol * scipy.random.normal()
            move3 = s3 * vol * scipy.random.normal()
            s1 += move1 
            s2 += move2
            s3 += move3
            
            move4 = corr12*move1 + math.sqrt(1-corr12*corr12)*move2
            s4 += move4
            
            move5 = corr13*move1+corr23*move4+ math.sqrt(1-(corr13*corr13)-(corr23*corr23))*move3
            s5 += move5       
            
            if(i%252 == 251):
                minimum = min(min(s1,s4),s5)
                if(minimum < 60):
                    knockin_indicator = 1
                elif(minimum > coupon_threshold):
                    coupon_num += 1
                
        minimum = min(min(s1,s4),s5)
        if(knockin_indicator == 1):
            sum += minimum
            sum += coupon_num*counpon_rate*stockpr
        else:
            sum += stockpr
            sum += coupon_num*counpon_rate*stockpr
            
    print("avg payout = " + str(sum/rounds))
    print("option price = " + str(sum/rounds-stockpr))

In [105]:
knockinPutBasket(100, 60, 1, 0.16, 10000, 0.1, 0.1, 0.1)
#correlations + 50%, +70%, +80%
knockinPutBasket(100, 60, 1, 0.16, 10000, 0.1*1.5, 0.1*1.5, 0.1*1.5)
knockinPutBasket(100, 60, 1, 0.16, 10000, 0.1*1.7, 0.1*1.7, 0.1*1.7)
knockinPutBasket(100, 60, 1, 0.16, 10000, 0.1*1.8, 0.1*1.8, 0.1*1.8)

avg payout = 107.46365988422329
option price = 7.46365988422329
avg payout = 107.43439762853966
option price = 7.4343976285396565
avg payout = 107.4568108617564
option price = 7.456810861756395
avg payout = 107.45276828825355
option price = 7.4527682882535515


In [106]:
#check the sensitivity to correlation
knockinPutBasket(100, 60, 1, 0.16, 10000, 0.11, 0.1, 0.1)
knockinPutBasket(100, 60, 1, 0.16, 10000, 0.11, 0.11, 0.11)
#sensitivity to correlation after changing the strike to 90%
knockinPutBasket(100, 90, 1, 0.16, 10000, 0.1, 0.1, 0.1)
knockinPutBasket(100, 90, 1, 0.16, 10000, 0.11, 0.1, 0.1)
knockinPutBasket(100, 90, 1, 0.16, 10000, 0.11, 0.11, 0.11)


avg payout = 107.37734384688227
option price = 7.377343846882269
avg payout = 107.4575454673877
option price = 7.457545467387703
avg payout = 107.41887411194158
option price = 7.418874111941577
avg payout = 107.43161191267534
option price = 7.431611912675336
avg payout = 107.41774178218157
option price = 7.417741782181565


In [107]:
knockinPutBasket(100, 60, 1, 0.16, 10000, 0.3, 0.3, 0.3)

#########
#Conclusion: down-and-in put on worst of 3 longs correlation
#########

avg payout = 107.53531713839801
option price = 7.535317138398014
